In [ ]:
# cd to yolov5 folder
%cd ~/project/yolov5-master

In [ ]:
# Required imports
import os 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import patches, text, patheffects
import torch
import cv2
import numpy as np
from IPython.display import Image, clear_output
import random
import shutil  

In [ ]:
# 
clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
def plot_sample_image(img_path, corr_lab, incorr_lab, miss_lab, gt_lab):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    fig, ax = plt.subplots()
    ax.imshow(img)
    plt.axis('off')

    for box in corr_lab:
        x = (box[0]-box[2]*0.5)*img.shape[1]
        y = (box[1]-box[3]*0.5)*img.shape[0]
        w = box[2] * img.shape[1]
        h = box[3] * img.shape[0]
        ax.add_patch(patches.Rectangle((x,y),w,h, fill=False, edgecolor='green', lw=1))

    for box in incorr_lab:
        x = (box[0]-box[2]*0.5)*img.shape[1]
        y = (box[1]-box[3]*0.5)*img.shape[0]
        w = box[2] * img.shape[1]
        h = box[3] * img.shape[0]
        ax.add_patch(patches.Rectangle((x,y),w,h, fill=False, edgecolor='red', lw=1))

    for box in miss_lab:
        x = (box[0]-box[2]*0.5)*img.shape[1]
        y = (box[1]-box[3]*0.5)*img.shape[0]
        w = box[2] * img.shape[1]
        h = box[3] * img.shape[0]
        ax.add_patch(patches.Rectangle((x,y),w,h, fill=False, edgecolor='blue', lw=1))

    fn = img_path.split('/')[-1].split('.')[0]
    plt.savefig('/home/ubuntu/project/Code/out/' + fn + '.png',dpi=600)
    #plt.show()

In [ ]:
PRED_img_path = "/home/ubuntu/project/yolov5-master/runs/detect/exp183/"
PRED_label_path = '/home/ubuntu/project/yolov5-master/runs/detect/exp183/labels/'

GT_label_path = "/home/ubuntu/project/sea_otter_dataset/dataset_otters_clusters_background/labels/validation/"
GT_img_path = "/home/ubuntu/project/sea_otter_dataset/dataset_otters_clusters_background/images/validation/"

In [ ]:
def xywhn2xyxy(x, w=1024, h=682):
    # Convert nx4 box from [x, y, w, h] normalized to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    x = np.asarray(x)
    y = np.copy(x)
    y[0] = w * (x[0] - x[2] / 2)  # top left x
    y[1] = h * (x[1] - x[3] / 2)  # top left y
    y[2] = w * (x[0] + x[2] / 2)  # bottom right x
    y[3] = h * (x[1] + x[3] / 2)  # bottom right y
    return y

In [ ]:
def xyxy2xywhn(x, w=1024, h=682):
    # Convert nx4 box from [x1, y1, x2, y2] to [x, y, w, h] normalized where xy1=top-left, xy2=bottom-right
    x = np.asarray(x)
    y = np.copy(x)
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[0] = ((x[0] + x[2]) / 2) / w  # x center
    y[1] = ((x[1] + x[3]) / 2) / h  # y center
    y[2] = (x[2] - x[0]) / w  # width
    y[3] = (x[3] - x[1]) / h  # height
    return y

In [ ]:
def box_iou(box, gtBoxes):
    """
    Compute max intersection over union of a bounding box and 
    a list of bounding boxes
    Arguments:
      boxA : np.array of shape (1,4)
      boxB : np.array of shape (1,4)
    Returns:
      iou : float
    """
    max_iou = 0

    for b in gtBoxes:
        xA = max(box[0], b[0])
        yA = max(box[1], b[1])
        xB = min(box[2], b[2])
        yB = min(box[3], b[3])

        iA = abs(max((xB - xA, 0)) * max((yB - yA), 0))
        if iA == 0:
            continue

        a1 = abs((box[2] - box[0]) * (box[3] - box[1]))
        a2 = abs((b[2] - b[0]) * (b[3] - b[1]))

        iou = iA / float(a1 + a2 - iA)
        if iou  > max_iou:
            max_iou = iou

    return max_iou

### Confusion Matrix + Image Analysis

In [ ]:
TP, TN, FP, FN = 0, 0, 0, 0
iou_th = 0.2
total_pred_labels = 0
total_gt_labels = 0

# Iterate on all images
for img in os.listdir(GT_img_path):
    if '.jpeg' not in img and '.png' not in img:
        continue
    txt = img.split('.')[0] + '.txt'

    true_labels = open(GT_label_path + txt,'r').read().splitlines()
    total_gt_labels += len(true_labels)
    try:
        pred_labels = open(PRED_label_path + txt,'r').read().splitlines()
        total_pred_labels += len(pred_labels)
    except:
        # Labels in ground truth but not in predicted
        FN += len(true_labels)
        continue
    tl = []
    for lab in true_labels:
        tl.append(xywhn2xyxy(list(map(float,lab.split(' ')[1:]))))

    pl = []
    for lab in pred_labels:
        pl.append(xywhn2xyxy(list(map(float,lab.split(' ')[1:]))))

    corr = []
    incorr = []
    miss = []
    gt = []
    for lab in pl:
        iou = box_iou(lab, tl)
        if iou > iou_th:
            corr.append(xyxy2xywhn(lab))
            TP += 1
        else:
            incorr.append(xyxy2xywhn(lab))
            FP += 1

    for lab in tl:
        iou = box_iou(lab, pl)
        gt.append(xyxy2xywhn(lab))
        if iou < iou_th:
            miss.append(xyxy2xywhn(lab))
            FN += 1

    if len(incorr) != 0 or len(miss) != 0:
        plot_sample_image(GT_img_path + img, corr, incorr, miss, gt)

In [ ]:
cm_data = [[TP, FP], [FN, TN]]
print('Precision', TP/(TP + FP))
print('Recall', TP/(TP + FN))
sns.heatmap(cm_data, annot=True, cmap='Blues', fmt='d', xticklabels=['Otters', 'Background'], yticklabels=['Otters', 'Background'])
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()

### Create Image Clusters

In [ ]:
label_path = "/content/drive/MyDrive/SeaOtterDrone2021/sea_otter_dataset/dataset_otters_clusters/labels/"
img_path = "/content/drive/MyDrive/SeaOtterDrone2021/sea_otter_dataset/dataset_otters_clusters/images/"

In [ ]:
!pip install -q image-similarity-measures

In [ ]:
# Idea from https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 
from keras.applications.vgg16 import VGG16 
from keras.models import Model
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
def extract_features(file, model):
    img = load_img(file, target_size=(224,224))
    img = np.array(img) 
    reshaped_img = img.reshape(1,224,224,3) 
    imgx = preprocess_input(reshaped_img)
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [ ]:
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    files = groups[cluster]
    
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        img = load_img(img_path + file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')
    plt.show()
    print('__________________________________')

In [ ]:
data = {}

model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

for filename in os.listdir(img_path):
    if '.jpeg' in filename:
        data[filename] = extract_features(img_path + filename, model)

filenames = np.array(list(data.keys()))
feat = np.array(list(data.values()))
feat = feat.reshape(-1,4096)

pca = PCA(n_components=50, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

sse = []
list_k = list(range(3, 50))

for k in list_k:
    km = KMeans(n_clusters=k, random_state=22, n_jobs=-1)
    km.fit(x)
    sse.append(km.inertia_)

plt.figure(figsize=(6, 6))
plt.plot(list_k, sse)
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

In [ ]:
kmeans = KMeans(n_clusters=50,n_jobs=-1, random_state=22)
kmeans.fit(x)

groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
def most_common(lst):
    return max(set(lst), key=lst.count)

In [ ]:
to_modify = []
for k,v in groups.items():
    print('Cluster:', k, len(v))
    view_cluster(k)
    if len(v) > 1:
        lab_size = []
    for im in v:
        labels = open(label_path + im.split('.')[0]+'.txt','r').read().splitlines()
        lab_size.append(len(labels))
    mc = most_common(lab_size)
    for i,s in enumerate(lab_size):
        print(mc, s, v[i])
        if s < mc:
            to_modify.append((mc-s,v[i]))

In [ ]:
cl_len = []
for k,v in groups.items():
    cl_len.append(len(v))
print(np.mean(cl_len), len(cl_len))
print(cl_len)

In [ ]:
random.seed(6)
non_train_cl = random.sample(range(0, 50), 8)
val_cl = non_train_cl[:4]
test_cl = non_train_cl[4:]

train_cl = list(set(np.arange(0,50))-set(val_cl)-set(test_cl))
print(val_cl)
print(test_cl)
print(len(train_cl),train_cl)

s = 0
for c in val_cl:
    s += len(groups[c])
print(s)

s = 0
for c in test_cl:
    s += len(groups[c])
print(s)

In [ ]:
train_path = "train/"
val_path = "validation/"
test_path = "test/"

for c in train_cl:
    for filename in groups[c]:
        shutil.move(img_path + filename, img_path + train_path)

for c in val_cl:
    for filename in groups[c]:
        shutil.move(img_path + filename, img_path + val_path)

for c in test_cl:
    for filename in groups[c]:
        shutil.move(img_path + filename, img_path + test_path)    

In [ ]:
train_path = "train/"
val_path = "validation/"
test_path = "test/"

for c in train_cl:
    for filename in groups[c]:
        filename = filename.split('.')[0] + '.txt'
        shutil.move(label_path + filename, label_path + train_path)

for c in val_cl:
    for filename in groups[c]:
        filename = filename.split('.')[0] + '.txt'
        shutil.move(label_path + filename, label_path + val_path)

for c in test_cl:
    for filename in groups[c]:
        filename = filename.split('.')[0] + '.txt'
        shutil.move(label_path + filename, img_path + test_path)   